<a href="https://colab.research.google.com/github/PollyBecker/NLP/blob/main/NER_Reconhecimento_de_Entidades_Nomeadas_e_criando_pad%C3%B5es_de_Match_%7C_spaCy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NER Named Entity Recognition
Named Entity Recognition (NER), em Português, Reconhecimento de Entidade Nomeada, é um processo de identificação de entidades predefinidas presentes em um texto, como nome da pessoa, organização, local que já vem pré-definidas no framework spaCy ou entidades que podemos criar com ajuda desta ferramenta.
# Matcher
O Matcher é uma ferramenta no spaCy que permite encontrar sequências de tokens em um documento com base em padrões que você define. Você pode criar padrões usando atributos como texto, formato de caracteres, parte do discurso, entre outros. O Matcher é útil para identificar padrões específicos no texto, como endereços de e-mail, números de telefone, ou qualquer outro tipo de estrutura que você deseja extrair automaticamente de um texto.

# 1- Instalando o SpaCy

https://www.youtube.com/watch?v=2XUhKpH0p4M

https://www.youtube.com/watch?v=9mXoGxAn6pM
https://spacy.io/models

In [ ]:
!pip install -U spacy

# 2- Fazendo o dowload do modelo


In [ ]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 19.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [18]:
import spacy
import pt_core_news_sm
nlp = pt_core_news_sm.load()

In [19]:
texto = nlp('Maria mora em Paris')

# Printando as entidades padão do modelo
for entidade in texto.ents:
  print(entidade.text, entidade.label_)

Maria PER
Paris LOC


In [20]:
# Visualização das entidades na frase
spacy.displacy.render(texto, style='ent', jupyter=True)

In [21]:
# O que é cada tipo de entidade(ver na doc do spacy)
print("PER:", spacy.explain('PER'))
print("LOC:", spacy.explain('LOC'))
print("ORG:", spacy.explain('ORG'))
print("MISC:", spacy.explain('MISC'))

PER: Named person or family.
LOC: Non-GPE locations, mountain ranges, bodies of water
ORG: Companies, agencies, institutions, etc.
MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art


In [22]:
text = nlp("Juliana nasceu no dia 18/06/2018 em Fortaleza")
spacy.displacy.render(text, style='ent', jupyter=True)

# 3- Criando um novo tipo de entidade no SpaCy
Aqui o modelo não identificou a data.

O spaCy permite re-treinar o modelo com novos dados melhorar o modelo e até gerar modelos novos.

Como a entidade data não foi identificada, podemos inserir alguns exemplos e re-treinar o algoritmo para que este aprenda novos padrões.

In [23]:
import random

In [27]:
TRAIN_DATA = [
    ("Almira é uma ótima pessoa, gosto muito dela", {"entities": [(0, 6, "PER")]}),
    ("João foi para Campo Grande nas férias", {"entities": [(14, 26, "LOC"), (0, 4, "PER")]}),
    ("Carlos foi visitar João na casa de praia", {"entities": [(0, 6, "PER"), (19, 23, "PER")]}),
    ("No meio do ano irei para São Paulo fazer mais um curso", {"entities": [(25, 34, "LOC")]}),
    ("O sonho dela era ir para Austrália visitar seu irmão", {"entities": [(25, 34, "LOC")]}),
    ("Em 15/07/1988 nasceu essa linda criança", {"entities": [(3, 13, "DATE"), (18, 24, "LOC")]}),
    ("Data de prisão: 10/01/2018", {"entities": [(16, 26, "DATE")]}),
    ("No dia 01/02/2016 foi decretada a sentença", {"entities": [(7, 17, "DATE")]}),
    ("A data da festa foi 07/05/2018", {"entities": [(20, 30, "DATE")]}),
    ("Dia 07/06/2020 choveu pela manhã", {"entities": [(4, 14, "DATE")]}),
    ("Michael Jackson nasceu dia 29/08/1958", {"entities": [(0, 15, "PER"), ( 27, 37, "DATE")]}),
    ("A viagem foi marcada para o dia 21/03/2021", {"entities": [(32, 42, "DATE")]}),
    ("O casamento, que era dia 12/06/2020, foi adiado para o dia 20/07/2020", {"entities": [(25, 35, "DATE"), (59, 68, "DATE")]}),
    ("Dia 21/04/2020 foi feriado", {"entities": [(4, 14, "DATE")]}),
    ("A entrega foi realizada na manhã do dia 18/03/2020 ", {"entities": [(40, 50, "DATE")]}),
    ("Na noite do dia 17/05/2019 aconteceu um eclipse", {"entities": [(16, 26, "DATE")]}),
    ("Rio de Janeiro é a capital do Rio de Janeiro", {"entities": [(0, 14, "LOC"), (30, 44, "LOC")]})
]

In [28]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

nlp = spacy.load("en_core_web_sm")

db = DocBin()

for text, annot in tqdm(TRAIN_DATA):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

os.chdir(r'/content')
db.to_disk("./train.spacy") # salvando o objeto

100%|██████████| 17/17 [00:00<00:00, 1224.53it/s]

Skipping entity
Skipping entity


In [29]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [30]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     59.64    0.00    0.00    0.00    0.00
148     200          9.23    803.14  100.00  100.00  100.00    1.00
348     400          0.00      0.00  100.00  100.00  100.00    1.00
548     600          0.00      0.00  100.00  100.00  100.00    1.00
748     800          0.00      0.00  100.00  100.00  100.00    1.00
948    1000          0.00      0.00  100.00  100.00  100.00    1.00
1148    1200          0.00      0.00  100.00  100.00  100.00    1.00
1348    1400          0.00      0.00  100.00  100.00  100.00   

In [32]:
nlp1 = spacy.load(r"/content/output/model-best")
doc = nlp1("Luana nasceu em Campo Grande em 31/05/2023")

spacy.displacy.render(doc, style="ent", jupyter=True)

# Matcher
Agora vamos usar o matcher para adicionar padrões que serão encontrados no texto de acondo com o shape do token.

In [31]:
from spacy.matcher import Matcher

texto = """
Meu número é (11) 98765 - 3333 ou sem espaço (11) 98765-4444 e o número do meu amigo é 55 11 8765 5555.
 Caso necessario entre em contato via email pelo miau@gmail.com.
"""
doc = nlp1(texto) # isso foi necessário pois o matcher le documentos ja processados pelo spacy e nao texto puro

matcher1 = Matcher(nlp1.vocab)

# Definir padrões para combinar números de telefone
patterns = [
    [{"TEXT": "("}, {"SHAPE": "dd"}, {"TEXT": ")"}, {"SHAPE": "dddd"}, {"TEXT": "-"}, {"SHAPE": "dddd"}],  # (11) 9876-5555
    [{"TEXT": "("}, {"SHAPE": "dd"}, {"TEXT": ")"},{"SHAPE": "dddd-dddd"}],  # (11) 98765-4444
    [{"SHAPE": "dd"}, {"SHAPE": "dd"}, {"SHAPE": "dddd"}, {"SHAPE": "dddd"}],   # 55 11 8765 5555

]
# Definir padrões para emails
email_pattern = [{'TEXT': {'REGEX': '[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}'}},]
matcher1.add('Regra A', [email_pattern])



# Adicionar os padrões ao matcher
for pattern in patterns:
    matcher1.add("PHONE_NUMBER", [pattern])

# Dando uma olhada no shape_
matches = matcher1(doc)
for token in doc:
    print(token.text, token.shape_,"---------------")
# Extrair e imprimir os números de telefone correspondentes
phone_numbers = [doc[start:end].text for match_id, start, end in matches]
print("Matches extraídos:", phone_numbers)



 
 ---------------
Meu Xxx ---------------
número xxxx ---------------
é x ---------------
( ( ---------------
11 dd ---------------
) ) ---------------
98765 dddd ---------------
- - ---------------
3333 dddd ---------------
ou xx ---------------
sem xxx ---------------
espaço xxxx ---------------
( ( ---------------
11 dd ---------------
) ) ---------------
98765-4444 dddd-dddd ---------------
e x ---------------
o x ---------------
número xxxx ---------------
do xx ---------------
meu xxx ---------------
amigo xxxx ---------------
é x ---------------
55 dd ---------------
11 dd ---------------
8765 dddd ---------------
5555 dddd ---------------
. . ---------------

  
  ---------------
Caso Xxxx ---------------
necessario xxxx ---------------
entre xxxx ---------------
em xx ---------------
contato xxxx ---------------
via xxx ---------------
email xxxx ---------------
pelo xxxx ---------------
miau@gmail.com xxxx@xxxx.xxx ---------------
. . ---------------

 
 ---------------
Mat

In [54]:
nlp2 = spacy.load("pt_core_news_sm")
texto = """Ana comeu batatas.
Pedro foi ao cinema com Joana.
Ontem encontrei uma amiga no Shopping.
Ana irá viajar na próxima semana"""
doc = nlp2(texto)

In [68]:
for token in doc:
    print(token.text, token.pos_,token.morph)

Ana PROPN Gender=Fem|Number=Sing
comeu VERB Mood=Sub|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin
batatas NOUN Gender=Fem|Number=Plur
. PUNCT 

 SPACE 
Pedro PROPN Gender=Masc|Number=Sing
foi VERB Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin
ao ADP Definite=Def|Gender=Masc|Number=Sing|PronType=Art
cinema NOUN Gender=Masc|Number=Sing
com ADP 
Joana PROPN Gender=Fem|Number=Sing
. PUNCT 

 SPACE 
Ontem ADV 
encontrei VERB Mood=Ind|Number=Sing|Person=1|Tense=Past|VerbForm=Fin
uma DET Definite=Ind|Gender=Fem|Number=Sing|PronType=Art
amiga NOUN Gender=Fem|Number=Sing
no ADP Definite=Def|Gender=Masc|Number=Sing|PronType=Art
Shopping PROPN Gender=Masc|Number=Sing
. PUNCT 

 SPACE 
Ana PROPN Gender=Fem|Number=Sing
irá AUX Mood=Ind|Number=Sing|Person=3|Tense=Fut|VerbForm=Fin
viajar VERB VerbForm=Inf
na ADP Definite=Def|Gender=Fem|Number=Sing|PronType=Art
próxima ADJ Gender=Fem|Number=Sing
semana NOUN Gender=Fem|Number=Sing


In [62]:
#Aqui vamos criar uma regra para pegar apenas a senteça realizada por uma mulher que esteja no passado
patern_past_fem = [{'POS':'PROPN', 'MORPH':{'IS_SUPERSET':["Gender=Fem"]}},
                   {'MORPH':{'IS_SUPERSET':['Tense=Pres']}},
                   {'OP':'?'},
                   {'POS':'NOUN'}
                   ]

In [63]:
matcher2 = Matcher(nlp2.vocab)
matcher2.add('Regra B', [patern_past_fem])

In [64]:
matches = matcher2(doc)
# Extrair e imprimir as sentenças correspondentes
for match_id, start, end in matches:
  print(doc[start:end].text)

Ana comeu batatas
